# Download Libraries

In [1]:
pip install psycopg2 pandas sqlalchemy pyarrow google-cloud-bigquery

Note: you may need to restart the kernel to use updated packages.


# Importing required libraries

In [2]:
import psycopg2
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

# Connecting to PostgreSQL from Python

In [3]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    dbname="weather_data",
    user="postgres",
    password="0595497329",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

print("Connected to PostgreSQL!")


Connected to PostgreSQL!


# Fetch data from PostgreSQL

In [4]:
query = "SELECT * FROM weather;"
weather_data = pd.read_sql(query, conn)

print("Fetched data from PostgreSQL:")
print(weather_data.head())

Fetched data from PostgreSQL:
   id  temperature  humidity  wind_speed  cloud_cover  pressure     rain
0   1        23.72     89.59        7.33        50.50   1032.37     rain
1   2        27.87     46.48        5.95         4.99    992.61  no rain
2   3        25.06     83.07        1.37        14.85   1007.23  no rain
3   4        23.62     74.36        7.05        67.25    982.63     rain
4   5        20.59     96.85        4.64        47.67    980.82  no rain


C:\Users\Anas\AppData\Local\Temp\ipykernel_28516\479444775.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  weather_data = pd.read_sql(query, conn)


# Load data from CSV

In [5]:
data = pd.read_csv(r'weather_forecast_data.csv')

# Data Cleaning and Transformation

## Getting some info about the data

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Temperature  2500 non-null   float64
 1   Humidity     2500 non-null   float64
 2   Wind_Speed   2500 non-null   float64
 3   Cloud_Cover  2500 non-null   float64
 4   Pressure     2500 non-null   float64
 5   Rain         2500 non-null   object 
dtypes: float64(5), object(1)
memory usage: 117.3+ KB


## Dropping rows with missing values

In [7]:
data = data.dropna()

## Transformation

In [8]:
data['Temperature'] = data['Temperature'].apply(lambda x: round(x, 2))  # Example transformation
print("Data cleaned successfully.")

Data cleaned successfully.


## Statistical measures about the data

In [9]:
data.describe()

,Temperature,Humidity,Wind_Speed,Cloud_Cover,Pressure
count,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000
mean,22.581772,64.347094,9.906255,49.658104,1014.312336
std,7.326991,19.954739,5.780316,29.123104,20.196433
min,10.000000,30.005071,0.009819,0.015038,980.014486
25%,16.360000,47.339815,4.761909,23.900016,996.938630
50%,22.535000,63.920797,9.908572,49.488284,1013.433035
75%,28.972500,81.561021,14.948408,75.324140,1031.735067
max,35.000000,99.997481,19.999132,99.997795,1049.985593


## Getting types for each feature

In [10]:
data.dtypes

Temperature    float64
Humidity       float64
Wind_Speed     float64
Cloud_Cover    float64
Pressure       float64
Rain            object
dtype: object

## Number of rows and columns in the dataset

In [11]:
data.shape

(2500, 6)

## Checking for missing values

In [12]:
data.isnull().sum()

Temperature    0
Humidity       0
Wind_Speed     0
Cloud_Cover    0
Pressure       0
Rain           0
dtype: int64

## Load data into PostgreSQL

In [13]:
for index, row in data.iterrows():
    cursor.execute("""
        INSERT INTO weather (temperature, humidity, wind_speed, cloud_cover, pressure, rain)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (row['Temperature'], row['Humidity'], row['Wind_Speed'], row['Cloud_Cover'], row['Pressure'], row['Rain']))

In [14]:
conn.commit()
print("Data uploaded to PostgreSQL successfully!")

Data uploaded to PostgreSQL successfully!


# Load data to Google BigQuery

In [15]:
print("Loading data to Google BigQuery...")
credentials = service_account.Credentials.from_service_account_file('weather-448908-24a8af7113c4.json')
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

Loading data to Google BigQuery...


# Define dataset and table in BigQuery

In [16]:
dataset_id = 'weather_data'
table_id = 'weather'

# Convert DataFrame to BigQuery format

In [17]:
!pip install pandas-gbq

In [ ]:
conda install -c conda-forge pandas-gbq

In [ ]:
data.to_gbq(destination_table=f'{dataset_id}.{table_id}', project_id=credentials.project_id, if_exists='replace', credentials=credentials)
print("Data loaded to Google BigQuery successfully!")

# Step 7: Close the connection

In [ ]:
cursor.close()
conn.close()
print("ETL pipeline completed successfully!")